### TO DO - Aug 22th
* solve the issue of calculation of the number of pads and number of drops, number of crops. 

In [1]:
#imports 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import urllib.parse
from glob import glob
import os
import ntpath 
import zipfile

PPMI_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/'

In [2]:
def reading_csv_files(folder_path):
    # saving all the csv files in a dataframe
    import os
    import glob
    # glob.glob('*.csv') #find all the csv files in a pathname. 
    os.chdir(folder_path)
    csv_files = [i for i in glob.glob('*.csv')]

    # Reading each csv file and storing them in a dictionnary containing the file name and the dataframe
    dict_files={}
    files_names=[]
    for files in csv_files:
        df=pd.read_csv(files,engine='python')
        basepath, filename=ntpath.split(files)
        files_names.append(filename)
        dict_files[filename]=df

    # Calculating the # of rows and features for each dataframe and storing them in a list of tuples
    shape_df=[]
    for i in range(0,len(dict_files.keys())):
        nrows, ncols=(len(dict_files[files_names[i]]),len(dict_files[files_names[i]].columns))
        shape_df.append((nrows,ncols))

    return (shape_df,files_names,dict_files)

In [3]:
def number_of_patients(df):
    '''return the number of patients in each csv files'''
    if 'PATNO' in df.columns:
        patients=len(list(set(df['PATNO'])))
    else: 
        patients=0
    return patients

In [4]:
def number_of_events(df): 
    '''return the number of events covered by each csv file'''
    if 'EVENT_ID' in df.columns:
        events=len(list(set(df['EVENT_ID'])))
    else: 
        events=0
    return events

In [5]:
def features_selection(list_df_sel):
    '''do features selection for each df,sel in list_df_sel'''
    return df_sel

In [112]:
def patients_selection(df, threshold=6):
    '''return a dataframe containing only the patients with number of visits > threshold'''
    if 'PATNO' in df.columns: 
        visits_number_by_pat=df.groupby('PATNO').size().sort_values(ascending=False)
        mask_sel=visits_number_by_pat>=threshold
        patients_sel=list(mask_sel[mask_sel.values==True].index)
        df=df.loc[df['PATNO'].isin(patients_sel),:]
        df=df.sort_values('PATNO')
        return (patients_sel, df)
    else:
        return 0

In [ ]:
def padding_cropping_analysis(df,input_timesteps=4): 
    visits_number=df.groupby('PATNO').size().sort_values(ascending=False)
    mask_crop=visits_number_by_pat>input_sequence
    mask_pad=(visits_number_by_pat<input_sequence) & (visits_number_by_pat>=(input_sequence-2))
    visitsvisits_number.where(visits_number > input_timesteps, )

In [7]:
def PD_patients_selection(df):
    folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/1-Subject-Characteristics/'
    files_names=reading_csv_files(folder_path)[1]
    dict_files=reading_csv_files(folder_path)[2]

    ### Patient_Status - actually useful for selecting only the PD patients in each table
    PRODROMA=dict_files[files_names[0]]
    PD_cat=['PD','REGPD','GENPD']
    PD_patients=list(PRODROMA.loc[PRODROMA['RECRUITMENT_CAT'].isin(PD_cat),'PATNO'])
    sub_df=df.loc[df['PATNO'].isin(PD_patients),:]
    return (PD_patients,sub_df)

In [31]:
def table_analysis(list_df,df_names,threshold=6,input_timesteps=4): 
    '''return a dataframe with: 
    -as rows: list of df
    -as columns: 
        - total number of observations=rows
        - total number of features=columns after features_selection - features_selection function 
        - number of events
        - number of PD patients by using PRODROMA info - number_of_patients function
        - number of patients with # of visits > threshold - number_of_patients function
        - shape of final df after selections of rows and columns
        -number of NaN values in the final df
        NB: takes as input the df with features selection. 
    '''   
    # initialisation df
    col=['observations_tot','features','events_num','patients_num','patients_sel','len_df_sel','NaN_values']
    df_df=pd.DataFrame(columns=col,index=df_names)

    # remplissage of dataframe for each row
    for i,df in enumerate(list_df): 
        df_df.iloc[i,:]['observations_tot']=len(df)
        df_df.iloc[i,:]['features']=len(df.columns)
        df_df.iloc[i,:]['events_num']=number_of_events(df)
        df_df.iloc[i,:]['patients_num']=number_of_patients(df)
        
        if not patients_selection(df)==0:
            df=patients_selection(df,threshold=threshold)[1]
            #df=PD_patients_selection(df)[1]          
            df_df.iloc[i,:]['patients_sel']=len(set(df['PATNO']))
        else: 
            if 'PATNO' in df.columns:
                #df=PD_patients_selection(df)[1]
                df_df.iloc[i,:]['patients_sel']=len(set(df['PATNO']))
            else:
                df_df.iloc[i,:]['patients_sel']=0            
        df_df.iloc[i,:]['len_df_sel']=len(df)
        df_df.iloc[i,:]['NaN_values']=(df.isnull().sum().sum())/(df_df.iloc[i,:]['len_df_sel']*df_df.iloc[i,:]['features'])
        
    return df_df

In [9]:
def INFODT_date(df):
    if 'INFODT' in df.columns: 
        df['INFODT_date']=df['INFODT'].apply(lambda x: datetime.strptime(x,'%m/%Y'))
    return df

## 1-Subject-Characteristics

In [10]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/1-Subject-Characteristics/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### Patient_Status - actually useful for selecting only the PD patients in each table
PRODROMA=dict_files[files_names[0]]
PD_cat=['PD','REGPD','GENPD']
PD_patients=list(PRODROMA.loc[PRODROMA['RECRUITMENT_CAT'].isin(PD_cat),'PATNO'])
print('number of PD patients:{}'.format(len(PD_patients)))


### Screening__Demographics - useful
#### NOT TS DATA
SCREEN=dict_files[files_names[3]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME',
       'CONSNTDT', 'APPRDX', 'CURRENT_APPRDX', 'P3GRP', 'BIRTHDT', 'GENDER',
       'HISPLAT', 'RAINDALS', 'RAASIAN', 'RABLACK', 'RAHAWOPI', 'RAWHITE',
       'RANOS', 'PRJENRDT']
SCREEN_sel=SCREEN[SEL]
print(number_of_patients(SCREEN))

number of PD patients:1052
2144


## 5-Medical History
### PDFEAT is important - keep it.

In [11]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/5-Medical-History/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'Diagnostic_Features.csv': useful: Feat-PD - 1670 patients.

DIAGFEAT=dict_files[files_names[1]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'DFSTROKE', 'DFRSKFCT', 'DFPRESNT', 'DFRPROG', 'DFSTATIC', 'DFHEMPRK',
       'DFAGESX', 'DFOTHCRS', 'DFCRSCM', 'DFRTREMP', 'DFRTREMA', 'DFPATREM',
       'DFOTHTRM', 'DFTREMCM', 'DFRIGIDP', 'DFRIGIDA', 'DFAXRIG', 'DFUNIRIG',
       'DFTONE', 'DFOTHRIG', 'DFRIGCM', 'DFBRADYP', 'DFBRADYA', 'DFAKINES',
       'DFBRPLUS', 'DFOTHABR', 'DFABRCM', 'DFPGDIST', 'DFGAIT', 'DFFREEZ',
       'DFFALLS', 'DFOTHPG', 'DFPGCM', 'DFPSYCH', 'DFCOGNIT', 'DFDYSTON',
       'DFCHOREA', 'DFMYOCLO', 'DFOTHHYP', 'DFHYPCM', 'DFHEMTRO', 'DFPSHYPO',
       'DFSEXDYS', 'DFURDYS', 'DFBWLDYS', 'DFOCULO', 'DFEYELID', 'DFNEURAB',
       'DFDOPRSP', 'DFRAPSPE', 'DFBULBAR', 'DFCTSCAN', 'DFMRI', 'DFATYP']
DIAGFEAT_sel=DIAGFEAT[SEL]

### 'General_Medical_History.csv': useful: Feat-Gen - 2067 patients
#### NOT TS DATA...

MHXGEN=dict_files[files_names[3]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'MHCAT', 'MHROW', 'MHHX', 'MHACTRES', 'MHDIAGYR', 
     'PT_CODE', 'PT_NAME', 'HLT_NAME', 'SOCABBR1']
# I need probably to select less features here
MHXGEN_sel=MHXGEN[SEL]

### General_Physical_Exam: useful: Feat-Gen - 2047 patients
GENPHYEX=dict_files[files_names[5]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PESEQ', 'PECAT', 'ABNORM']
GENPHYEX_sel=GENPHYEX[SEL]

### 'General_Neurological_Exam.csv': useful: Feat-Gen - 2048 patients
PENEURO=dict_files[files_names[7]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'MSRARSP', 'MSRACM', 'MSLARSP', 'MSLACM', 'MSRLRSP', 'MSRLCM',
       'MSLLRSP', 'MSLLCM', 'COFNRRSP', 'COFNRCM', 'COFNLRSP', 'COFNLCM',
       'COHSRRSP', 'COHSRCM', 'COHSLRSP', 'COHSLCM', 'SENRARSP', 'SENRACM',
       'SENLARSP', 'SENLACM', 'SENRLRSP', 'SENRLCM', 'SENLLRSP', 'SENLLCM',
       'RFLRARSP', 'RFLRACM', 'RFLLARSP', 'RFLLACM', 'RFLRLRSP', 'RFLRLCM',
       'RFLLLRSP', 'RFLLLCM', 'PLRRRSP', 'PLRRCM', 'PLRLRSP', 'PLRLCM']
PENEURO_sel=PENEURO[SEL]

### 'Neurological_Exam_-_Cranial_Nerves.csv': useful: Feat-PD - 2049 patients
PENEURO2=dict_files[files_names[13]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'CN1RSP','CN2RSP', 'CN346RSP','CN5RSP',
     'CN7RSP', 'CN8RSP', 'CN910RSP',
       'CN11RSP', 'CN12RSP']
PENEURO2_sel=PENEURO2[SEL]

### 'PD_Features.csv': useful: Feat-PD - caution: these are actually not timeseries but examination only done at screening (SC), BL & V04
### 1017 patients
PDFEAT=dict_files[files_names[14]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT', 'SXMO',
       'SXYEAR', 'PDDXDT', 'DXTREMOR', 'DXRIGID', 'DXBRADY',
       'DXPOSINS', 'DXOTHSX','DOMSIDE']
PDFEAT_sel=PDFEAT[SEL]
#list(set(PDFEAT_sel['DXOTHCM'])) - see if we remove the comment feature or not.

### 'Vital_Signs.csv': useful: Feat-Gen
### 2047 patients
VITAL=dict_files[files_names[15]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'WGTKG', 'HTCM', 'TEMPC', 'SYSSUP', 'DIASUP', 'HRSUP',
       'SYSSTND', 'DIASTND', 'HRSTND']
VITAL_sel=VITAL[SEL]

In [12]:
# merging the 2 dataframes. 
PD_features=pd.merge(DIAGFEAT_sel,PDFEAT_sel,how='left',on=['PATNO','EVENT_ID','INFODT'])
len(PD_features)
# merge MHXGEN & GENPHYEX
Gen_Medical=pd.merge(MHXGEN_sel,GENPHYEX_sel,how='left',on=['PATNO','EVENT_ID','INFODT'])
len(Gen_Medical)
Gen_Medical=pd.merge(Gen_Medical,VITAL_sel,how='left',on=['PATNO','EVENT_ID','INFODT'])
# merge PENNEURO/PENNEURO2
Gen_Neuro=pd.merge(PENEURO_sel,PENEURO2_sel,how='inner',on=['PATNO','EVENT_ID','INFODT'])
len(Gen_Neuro)

6217

## 6-Motor-Assessments - all of them can be useful. 
* UPDRS1 assesses non-motor experiences of daily-living 
* UPDRS2P assesses motor experiences of daily living
* UPDRS3 assesses motor signs of PD. Includes Hoehn & Yahr Stage. 
* UPDRS4 assesses motor complications, dyskinesias, & moto fluctuations using historical & objective information.

In [13]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/6-Motor-Assessments/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'MDS_UPDRS_Part_I__Patient_Questionnaire.csv' - 2011 patients
NUPDR1SP=dict_files[files_names[0]]

### 'MDS_UPDRS_Part_I.csv' - 2010 patients - 
NUPDRS1=dict_files[files_names[2]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'NP1COG', 'NP1HALL', 'NP1DPRS', 'NP1ANXS', 'NP1APAT',
       'NP1DDS']
NUPDRS1_sel=NUPDRS1[SEL]

### 'MDS_UPDRS_Part_II__Patient_Questionnaire.csv' - 2011 patients
NUPDRS2P=dict_files[files_names[11]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'NP2SPCH', 'NP2SALV', 'NP2SWAL', 'NP2EAT', 'NP2DRES',
       'NP2HYGN', 'NP2HWRT', 'NP2HOBB', 'NP2TURN', 'NP2TRMR', 'NP2RISE',
       'NP2WALK', 'NP2FREZ']
NUPDRS2P_sel=NUPDRS2P[SEL]

### 'MDS_UPDRS_Part_III.csv' - 2010 patients
NUPDRS3=dict_files[files_names[7]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'CMEDTM', 'NP3SPCH', 'NP3FACXP', 'NP3RIGN', 'NP3RIGRU',
       'NP3RIGLU', 'PN3RIGRL', 'NP3RIGLL', 'NP3FTAPR', 'NP3FTAPL', 'NP3HMOVR',
       'NP3HMOVL', 'NP3PRSPR', 'NP3PRSPL', 'NP3TTAPR', 'NP3TTAPL', 'NP3LGAGR',
       'NP3LGAGL', 'NP3RISNG', 'NP3GAIT', 'NP3FRZGT', 'NP3PSTBL', 'NP3POSTR',
       'NP3BRADY', 'NP3PTRMR', 'NP3PTRML', 'NP3KTRMR', 'NP3KTRML', 'NP3RTARU',
       'NP3RTALU', 'NP3RTARL', 'NP3RTALL', 'NP3RTALJ', 'NP3RTCON', 'DYSKPRES',
       'DYSKIRAT', 'NHY', 'ANNUAL_TIME_BTW_DOSE_NUPDRS', 'ON_OFF_DOSE',
       'PD_MED_USE']
NUPDRS3_sel=NUPDRS3[SEL]

### 'MDS_UPDRS_Part_IV.csv' - 860 patients
NUPDRS4=dict_files[files_names[5]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'NP4WDYSK', 'NP4DYSKI', 'NP4OFF', 'NP4FLCTI', 'NP4FLCTX', 'NP4DYSTN']
NUPDRS4_sel=NUPDRS4[SEL]

In [14]:
### 'PASE_-_Household_Activity.csv' - 1248 patients
PASE=dict_files[files_names[3]]

### 'PASE_-_Leisure_Time_Activity.csv' - 1248 patients
PASELT=dict_files[files_names[10]]

### 'Modified_Schwab_+_England_ADL.csv' - 1837 patients
MODSEADL=dict_files[files_names[6]]

In [15]:
NUPDRS1_2P=pd.merge(NUPDRS1_sel,NUPDRS2P_sel,how='inner',on=['PATNO','EVENT_ID','INFODT'])
NUPDRS1_2P.head()

,REC_ID_x,PATNO,EVENT_ID,PAG_NAME_x,INFODT,NP1COG,NP1HALL,NP1DPRS,NP1ANXS,NP1APAT,...,NP2EAT,NP2DRES,NP2HYGN,NP2HWRT,NP2HOBB,NP2TURN,NP2TRMR,NP2RISE,NP2WALK,NP2FREZ
0,231798601,3402,SC,NUPDRS1,07/2010,0.0,0.0,1.0,0.0,2.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,233164301,3400,BL,NUPDRS1,07/2010,0.0,0.0,2.0,3.0,2.0,...,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0
2,230955201,3403,SC,NUPDRS1,07/2010,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
3,234837501,3404,BL,NUPDRS1,07/2010,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,232712901,3406,SC,NUPDRS1,07/2010,0.0,0.0,0.0,1.0,1.0,...,2.0,0.0,1.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0


## 7-Non-motor-assessments - all of them can be useful but we might need to choose. 
### Do a correlation analysis and ask doctors which ones are the more relevant. 

In [16]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/7-Non-motor-assessments/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'University_of_Pennsylvania_Smell_ID_Test.csv' - 1799 patients - OLFACTION
UPSIT=dict_files[files_names[0]]
UPSIT.head()

### 'Epworth_Sleepiness_Scale.csv - 1343 patients - SLEEP. 
EPWORTH=dict_files[files_names[2]]
EPWORTH.head()

### 'Letter_-_Number_Sequencing__PD_.csv' - 1341 patients. - MEMORY. 
LNSPD=dict_files[files_names[3]]
LNSPD.head()

### 'Olfactory_UPSIT.csv' - 4468 patients - OLFACTION. 
#### NB: only take the 2 last columns as features (TOTAL_CORRECT, UPSIIT_PRCNTGE)
OLFACT=dict_files[files_names[4]]
OLFACT.head()

### 'REM_Sleep_Disorder_Questionnaire.csv' - 1810 patients - SLEEP. 
REMSLEEP=dict_files[files_names[5]]
REMSLEEP.head()

### 'Cognitive_Assessments.csv' - 1172 patients
COGTIME=dict_files[files_names[6]]
COGTIME.head()

### 'SCOPA-AUT.csv' - 1345 patients - AUTOMATIC DYSFUNCTIONS
SCOPAAUT=dict_files[files_names[7]]
SCOPAAUT.head()

### Cognitive_Categorization.csv' - 1302 patients - DEMENTIA/MILD COGNITIVE IMPAIRMENT
COGCATG=dict_files[files_names[8]]
COGCATG.head()

### 'Hopkins_Verbal_Learning_Test.csv' - 1341 patients - MEMORY
HVLT=dict_files[files_names[9]]
HVLT.head()

### 'Montreal_Cognitive_Assessment__MoCA_.csv' - 2054 patients - MILD COGNITIVE IMPAIRMENT - PD 
MOCA=dict_files[files_names[10]]
MOCA.head()

### 'Semantic_Fluency.csv' - 1342 patients - Lexical/Semantic Fluency
SFT=dict_files[files_names[11]]
SFT.head()

### 'Geriatric_Depression_Scale__Short_.csv' - 1478 patients - DEPRESSION
GDSSHORT=dict_files[files_names[12]]
GDSSHORT.head()

### 'Symbol_Digit_Modalities.csv' - 1340 patients - COGNITIVE IMPAIRMENT. 
SDM=dict_files[files_names[13]]
SDM.head()

### 'State-Trait_Anxiety_Inventory.csv' - 1474 patients - DEPRESSION
STAI=dict_files[files_names[14]]
STAI.head()

### 'Benton_Judgment_of_Line_Orientation.csv' - 1341 patients - SPATIAL PERCEPTION/ORIENTATION 
BENTONOD=dict_files[files_names[15]]
BENTONOD.head()

### 'QUIP_Current_Short.csv' - 1346 patients - COMPULSIVE DISORDERS
QUIPCS=dict_files[files_names[16]]

## Medications data

In [24]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/5-Medical-History/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

In [25]:
### Use of PD Medication: useful: Meds - but not to be used for now.
PDMEDUSE=dict_files[files_names[4]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PDMEDYN', 'ONLDOPA', 'ONDOPAG', 'ONOTHER', 'FULNUPDR', 'PDMEDDT',
       'PDMEDTM', 'NUPDRTM']
PDMEDUSE_sel=PDMEDUSE[SEL]

### 'Concomitant_Medications.csv': useful: Meds - no time to exploit this stuff. 
CMED=dict_files[files_names[6]]

### 'Current_Medical_Conditions_Log.csv': Meds
CURRCOND=dict_files[files_names[8]]
CURRCOND.columns
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'SEQNO',
       'CONDCAT', 'DIAGYR','CONDTERM', 'RESOLVD', 'RESYR',
       'HLGTNAME', 'HLT_NAME','PT_CODE','SOCABBR1', 'VMEDDRA']
CURRCOND_sel=CURRCOND[SEL]

### 'Surgery_for_Parkinson_Disease.csv': useful: Meds - but not to be used for now.
SURGPD=dict_files[files_names[12]]
SURGPD.columns
SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PDSURG', 'PDSURGDT', 'PDSURGTP']
SURGPD_sel=SURGPD[SEL]

In [44]:
list_df=[DIAGFEAT_sel,MHXGEN_sel,GENPHYEX_sel,PENEURO_sel,PENEURO2_sel,VITAL_sel,
NUPDRS1,NUPDRS2P,NUPDRS3,NUPDRS4,PASE,PASELT,MODSEADL,
UPSIT,EPWORTH,LNSPD,OLFACT,REMSLEEP,COGTIME,SCOPAAUT,COGCATG,HVLT,
MOCA,SFT,GDSSHORT,SDM,STAI,BENTONOD,QUIPCS,PDMEDUSE]

df_names=['DIAGFEAT','MHXGEN','GENPHYEX','PENEURO','PENEURO2','VITAL',
'NUPDRS1','NUPDRS2P','NUPDRS3','NUPDRS4','PASE','PASELT','MODSEADL',
'UPSIT','EPWORTH','LNSPD','OLFACT','REMSLEEP','COGTIME','SCOPAAUT','COGCATG','HVLT',
'MOCA','SFT','GDSSHORT','SDM','STAI','BENTONOD','QUIPCS','PDMEDUSE']

dataset_analysis_maintables6=table_analysis(list_df,df_names)
dataset_analysis_maintables6

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in long_scalars


,observations_tot,features,events_num,patients_num,patients_sel,len_df_sel,NaN_values
DIAGFEAT,5739,59,19,1670,320,2349,0.100779
MHXGEN,63933,14,2,2067,2067,63933,0.383794
GENPHYEX,24382,8,8,2047,2047,24382,0.000148675
PENEURO,6217,41,15,2048,440,2877,0.428563
PENEURO2,6218,14,15,2049,440,2877,0
VITAL,12762,14,21,2047,910,10544,0.0818407
NUPDRS1,11236,17,20,2010,856,8859,0.0588567
NUPDRS2P,11235,24,20,2011,857,8864,0.0418829
NUPDRS3,12696,51,20,2010,896,10440,0.0989219
NUPDRS4,4541,16,19,860,405,3464,0.0626443


In [45]:
dataset_analysis_maintables2=table_analysis(list_df,df_names,2)
dataset_analysis_maintables2

,observations_tot,features,events_num,patients_num,patients_sel,len_df_sel,NaN_values
DIAGFEAT,5739,59,19,1670,1174,5243,0.10101
MHXGEN,63933,14,2,2067,2067,63933,0.383794
GENPHYEX,24382,8,8,2047,2047,24382,0.000148675
PENEURO,6217,41,15,2048,1264,5433,0.428614
PENEURO2,6218,14,15,2049,1264,5433,0
VITAL,12762,14,21,2047,1445,12160,0.0796699
NUPDRS1,11236,17,20,2010,1433,10659,0.0588511
NUPDRS2P,11235,24,20,2011,1432,10656,0.0418426
NUPDRS3,12696,51,20,2010,1439,12125,0.100264
NUPDRS4,4541,16,19,860,673,4354,0.0628158


In [47]:
dataset_analysis_maintables4=table_analysis(list_df,df_names,4)
dataset_analysis_maintables4

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in long_scalars


,observations_tot,features,events_num,patients_num,patients_sel,len_df_sel,NaN_values
DIAGFEAT,5739,59,19,1670,717,4169,0.101065
MHXGEN,63933,14,2,2067,2067,63933,0.383794
GENPHYEX,24382,8,8,2047,2047,24382,0.000148675
PENEURO,6217,41,15,2048,723,4140,0.428467
PENEURO2,6218,14,15,2049,723,4140,0
VITAL,12762,14,21,2047,1089,11357,0.0818627
NUPDRS1,11236,17,20,2010,1071,9840,0.0588534
NUPDRS2P,11235,24,20,2011,1070,9836,0.0418573
NUPDRS3,12696,51,20,2010,1089,11322,0.0998403
NUPDRS4,4541,16,19,860,524,3998,0.0626251


In [105]:
sel_to_plot=['DIAGFEAT','GENPHYEX','NUPDRS3','MODSEADL','MOCA','PDMEDUSE']

df_to_plot4=dataset_analysis_maintables4.loc[sel_to_plot,['patients_sel','len_df_sel']]
col_renamed=dict(zip(list(df_to_plot4.columns),['len_patients4','len_df_sel4']))
df_to_plot4.rename(columns=col_renamed,inplace=True)

df_to_plot2=dataset_analysis_maintables2.loc[sel_to_plot,['patients_sel','len_df_sel']]
col_renamed=dict(zip(list(df_to_plot2.columns),['len_patients2','len_df_sel2']))
df_to_plot2.rename(columns=col_renamed,inplace=True)


df_to_plot6=dataset_analysis_maintables6.loc[sel_to_plot,['patients_sel','len_df_sel']]
col_renamed=dict(zip(list(df_to_plot6.columns),['len_patients6','len_df_sel6']))
df_to_plot6.rename(columns=col_renamed,inplace=True)

df_to_plot=pd.merge(df_to_plot2,df_to_plot4,left_index=True,right_index=True)
df_to_plot=pd.merge(df_to_plot,df_to_plot6,left_index=True,right_index=True)
df_to_plot=df_to_plot.T
df_to_plot['Measure']=list(np.tile(('len_patients','len_df_sel'),3))
df_to_plot['Visits_threshold']=list(np.repeat((2,4,6),2))


df_to_plot_len_patients=df_to_plot.loc[['len_patients2','len_patients4','len_patients6'],:]
df_to_plot_len_patients.drop(columns='Measure',inplace=True)
df_to_plot_len_patients.rename(index=dict(zip(list(df_to_plot_len_patients.index)
                                              ,list(df_to_plot_len_patients['Visits_threshold'].values))),
                               inplace=True)
df_to_plot_len_patients.drop(columns='Visits_threshold',inplace=True)
df_to_plot_len_patients

df_to_plot_len_samples=df_to_plot.loc[['len_df_sel2','len_df_sel4','len_df_sel6'],:]
df_to_plot_len_samples.drop(columns='Measure',inplace=True)
df_to_plot_len_samples.rename(index=dict(zip(list(df_to_plot_len_samples.index)
                                              ,list(df_to_plot_len_samples['Visits_threshold'].values))),
                               inplace=True)
df_to_plot_len_samples.drop(columns='Visits_threshold',inplace=True)
df_to_plot_len_samples

,DIAGFEAT,GENPHYEX,NUPDRS3,MODSEADL,MOCA,PDMEDUSE
2,5243,24382,12125,9606,5393,6518
4,4169,24382,11322,8811,4093,6294
6,2349,24382,10440,7985,2824,5873


In [109]:
import cufflinks as cf
# Configure it to work offline:
cf.go_offline(connected=True)

df_to_plot_len_patients.iplot(kind='bar',
                              bins=list(df_to_plot_len_patients.index),
                              title='number of patients per table for each number of visits threshold',
                              xTitle='number of visits thresold',
                            yTitle='number of patients per table')

In [113]:
df_to_plot_len_samples.iplot(kind='bar',
                              bins=list(df_to_plot_len_patients.index),
                              title='number of samples per table for each number of visits threshold',
                              xTitle='number of visits thresold',
                            yTitle='number of samples per table')

In [162]:
input_timesteps=4
visits_number=DIAGFEAT_sel.groupby('PATNO').size().sort_values(ascending=False)
df_visits=visits_number.to_frame().rename(columns=dict(zip([0],['visits_num'])))
#mask_crop=visits_number_by_pat>input_sequence
#mask_pad=(visits_number_by_pat<input_sequence) & (visits_number_by_pat>=(input_sequence-2))
df_visits['processing_op']=df_visits['visits_num'].where(visits_number > input_timesteps, other='padding')
#visits_number
df_visits['processing_op'].where(df_visits['processing_op']=='padding',other='cropping',inplace=True)

mask_drop=(df_visits['visits_num'] < (input_timesteps-2))
#mask_drop
df_visits[mask_drop].replace(to_replace='padding',value='dropping',inplace=True)

#mask_nothing=((df_visits['visits_num']==input_timesteps))
#df_visits.loc[mask_drop,'processing_op'].replace(to_replace='padding',value='nothing',inplace=True)

/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3795: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [163]:
df_visits

,visits_num,processing_op
PATNO,,
60044,11,cropping
14331,11,cropping
60101,11,cropping
60065,11,cropping
60063,11,cropping
60004,11,cropping
60059,11,cropping
60057,11,cropping
60107,11,cropping


In [141]:
for i,boolean in enumerate(mask_drop):
    if boolean==True:
           df_visits.loc[i,'processing_op']
           #row_to_drop['processing_op']='dropping'

#mask_pad=((df_visits['visits_num'] >=(input_timesteps-2)) & (df_visits['visits_num'] < input_timesteps))
           
#for i,boolean in enumerate(mask_pad):
    #if boolean==True:
          #df_visits.iloc[i]['processing_op']='padding'
        

mask_nothing=((df_visits['visits_num']==input_timesteps))
        
for i,boolean in enumerate(mask_nothing):
    if boolean==True:
        row_to_drop=df_visits.iloc[i,:]
        row_to_drop['processing_op']='dropping'
            
df_visits.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,visits_num,processing_op
PATNO,,
60044,11,cropping
14331,11,cropping
60101,11,cropping
60065,11,cropping
60063,11,cropping


In [142]:
df_visits[df_visits['visits_num']==4]

,visits_num,processing_op
PATNO,,
3275,4,padding
54908,4,padding
40757,4,padding
3604,4,padding
3787,4,padding
3609,4,padding
3612,4,padding
40756,4,padding
42006,4,padding


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(15,10))
for i, (name, row) in enumerate(df_to_plot4.iterrows()):
    ax = plt.subplot(2,3, i+1)
    ax.set_title(row.name)
    ax.set_aspect('equal')
    ax.plot()

plt.show()

# Tables to plot: 
* DIAGFEAT
* GENPHYEX
* NUPDRS3
* NUPDRS4
* MODSEADL
* MOCA
* PDMEDUSE

## 2-Biospecimen

In [18]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/2-Biospecimen/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### Genetic_Results_Tests - useful: Feat-bio - 1116 patients
#### NOT TS DATA

MUTRSLT=dict_files[files_names[3]]
SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'GENECAT', 'LRRKCD', 'MUTRSLT']
MUTRSLT_sel=MUTRSLT[SEL]
print(number_of_patients(MUTRSLT))
MUTRSLT_sel.head()

### Pilot_Biospecimen_Analysis_Results - useful: Feat-bio
#### Understand if the cohort of patient IDs are different from the classic PPMI cohort.
#### NOT TS DATA.

### Pilot_Biospecimen_Analysis_Results - useful: Feat-bio
Biospecimen_Analysis=dict_files[files_names[6]]

SEL=['PATNO', 'DIAGNOSIS', 'CLINICAL_EVENT', 'TYPE', 'TESTNAME',
       'TESTVALUE','RUNDATE', 'PROJECTID']
print(number_of_patients(Biospecimen_Analysis))
Biospecimen_Analysis_sel=Biospecimen_Analysis[SEL]

### Lumbar_Puncture_Sample_Collection - useful: Feat-bio - 1344 patients
LUMBAR=dict_files[files_names[9]]

### Only features useful I think: white cell blood count, red blood cell count, total glucose, total protein.

SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT', 'FASTSTAT','PDMEDYN', 'PDMEDDT', 'PDMEDTM',
     'SMPDSCRD','RBCRSLT','WBCRSLT','TOPRRSLT','TGLCRSLT','SPFI','FLUORO']
print(number_of_patients(LUMBAR))
#print(len(patients_selection(LUMBAR,6)[0]))
LUMBAR_sel=LUMBAR[SEL]

1116
664
1344


## 3-Enrollment - useless?

In [19]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/3-Enrollment/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### 'Primary_Diagnosis.csv' - useful: Feat-Gen - 789 patients... (only PD patients?)
PRIMDXPD=dict_files[files_names[8]]

SEL=['REC_ID', 'F_STATUS', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PRIMDIAG', 'OTHNEURO']
print(number_of_patients(PRIMDXPD))
PRIMDXPD_sel=PRIMDXPD[SEL]

files_names

789


['Florbetaben_Eligibility.csv',
 'Contact_Information_Brain_Bank.csv',
 'Conclusion_of_Study_Participation.csv',
 'ST_CATALOG.csv',
 'Skin_Biopsy_Eligibility.csv',
 'RBD_PSG_Eligibility.csv',
 'Inclusion_Exclusion.csv',
 'AV-133_Subject_Eligibility.csv',
 'Primary_Diagnosis.csv',
 'TAP-PD_Conclusion.csv',
 'Signature_Form.csv',
 'FOUND_Enrollment_Status.csv',
 'Contact_Information_FOUND.csv',
 'Research_Advance_Directive.csv',
 'TAP-PD_Subject_Eligibility.csv']

# Datasets to investigate

### Blood_Chemistry__Hematology - Contains information about patient blood. 
### Ask doctors if really useful. 

In [20]:
COVANCE=dict_files[files_names[12]]
#COVANCE.head()

### IUSM_Catalog - DNK - most likely useless
#### seems ti be only metadata on the lab collections.
Look @ the PPMI doc to understand what is this table

In [21]:
IUSMCAT=dict_files[files_names[14]]
#IUSMCAT.head()

# Additionnal data that will be used later on (imaging data and medication data)

## 4-Imaging - dataset not to be used for the time being.
### Info about Imaging Data pages 98 to 101 in the study protocol. 

In [22]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/4-Imaging/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

### FBB_Analysis_Data.csv
FBBANAL=dict_files[files_names[0]]
# no selection to do. 

### AV-133_Imaging.csv
AVIMAG=dict_files[files_names[2]]
SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT', 'PETCMPLT','VMATRSLT']
AVIMAG_sel=AVIMAG[SEL]

### DaTscan_Imaging.csv
DATSCAN=dict_files[files_names[3]]
DATSCAN.columns
SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'DATSCAN','VSINTRPT','VSRPTELG']
DATSCAN_sel=DATSCAN[SEL]

### AV-133_SBR_Results.csv
ind_av133_sbr=dict_files[files_names[5]]
col_old=list(ind_av133_sbr.columns)
col_new=['PATNO',
 'EVENT_ID',
 'INFODT',
 'TIMEPOINT',
 'RCAUD-S',
 'RPUTANT-S',
 'RPUTPOST-S',
 'LCAUD-S',
 'LPUTANT-S',
 'LPUTPOST-S',
 'update_stamp']
col=dict(zip(col_old,col_new))
ind_av133_sbr.rename(columns=col,inplace=True)
SEL=['PATNO', 'EVENT_ID', 'INFODT', 'RCAUD-S', 'RPUTANT-S',
       'RPUTPOST-S', 'LCAUD-S', 'LPUTANT-S', 'LPUTPOST-S']
ind_av133_sbr_sel=ind_av133_sbr[SEL]

### Magnetic_Resonance_Imaging.csv
MRI=dict_files[files_names[6]]
SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'MRICMPLT', 'MRIWDTI', 'MRIWRSS', 'MRIRSLT',
       'PDMEDYN', 'ONLDOPA', 'ONDOPAG']
MRI_sel=MRI[SEL]

### DTI_Regions_of_Interest
DTIROI=dict_files[files_names[7]]
SEL=['PATNO', 'PAG_NAME', 'INFODT', 'Measure','ROI1', 'ROI2',
       'ROI3', 'ROI4', 'ROI5', 'ROI6', 'REF1', 'REF2']
DTIROI_sel=DTIROI[SEL]

### DaTSCAN_SPECT_Visual_Interpretation_Assessment.csv - not found in data dictionary...
SPECT_vis_assess=dict_files[files_names[8]]
#SPECT_vis_assess.head()

In [34]:
list_df=[FBBANAL,AVIMAG_sel,DATSCAN_sel,ind_av133_sbr_sel,MRI_sel,DTIROI_sel]
df_names=['FBBANAL','AVIMAG','DATSCAN','ind_av133_sbr','MRI','DTIROI']
dataset_analysis_imaging=table_analysis(list_df,df_names,threshold=3) #threshold=3 to have more 
dataset_analysis_imaging

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:36: RuntimeWarning: invalid value encountered in long_scalars


,observations_tot,features,events_num,patients_num,patients_sel,len_df_sel,NaN_values
FBBANAL,88,37,6,88,0,0,NaN
AVIMAG,87,7,9,36,16,57,0.105263
DATSCAN,2483,8,16,1045,414,1637,0.213653
ind_av133_sbr,80,9,6,35,14,50,0
MRI,2125,12,16,1326,253,926,0.330274
DTIROI,1200,12,0,263,263,1200,0


In [35]:
list_df=[PDMEDUSE_sel,CURRCOND_sel,AE_sel,SURGPD_sel]
df_names=['PDMEDUSE','CURRCOND','AE','SURGPD']
dataset_analysis_meds=table_analysis(list_df,df_names)
dataset_analysis_meds

,observations_tot,features,events_num,patients_num,patients_sel,len_df_sel,NaN_values
PDMEDUSE,6668,13,21,909,576,5873,0.277764
CURRCOND,11489,15,3,1619,891,9262,0.0742676
AE,1243,25,3,620,28,193,0.170777
SURGPD,3403,8,23,896,224,1452,0.24759


### Family_History_PD - most likely useful but not to be used for now.
#### Can be useful for detecting early PD

In [27]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/1-Subject-Characteristics/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

FAMHXPD=dict_files[files_names[2]]
#FAMHXPD.head()

### 'Prodromal_Diagnostic_Questionnaire.csv': useful: Feat-Gen (PRIMDIAG & PSLVL2) 
#### NB: PSLV2 evaluates the proba of the patient being PD: might be useful for early PD. But not to be used in the main analysis as a feature (only for Prodromal & Genetic Cohort - i.e 800 subjects) 

In [28]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/5-Medical-History/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

PRODIAG=dict_files[files_names[0]]
SEL=['REC_ID', 'F_STATUS', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'PRIMDIAG',
       'OTHNEURO', 'PSLVL2']
PRODIAG_sel=PRODIAG[SEL]
PRODIAG_sel.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,PRIMDIAG,OTHNEURO,PSLVL2
0,408770801,V,10362,SC,PRODDIAG,17,NaN,4
1,605736601,V,10405,SC,PRODDIAG,17,NaN,5
2,410987801,V,10496,SC,PRODDIAG,23,NaN,5
3,409586301,S,10606,SC,PRODDIAG,17,NaN,4
4,448326801,V,10662,SC,PRODDIAG,17,NaN,5


### 'Signature_Form.csv' - most likely useful: Meds - but not to be used for now.
#### Can be useful for the meds log

In [29]:
folder_path='/Users/alicemartin/02_DSR_Project/parkinson-disease-project/data/PPMI-final-dataset-382018/3-Enrollment/'
files_names=reading_csv_files(folder_path)[1]
dict_files=reading_csv_files(folder_path)[2]

SIG=dict_files[files_names[10]]
SIG.head()

,REC_ID,F_STATUS,PATNO,EVENT_ID,PAG_NAME,INFODT,VISSTAT,VISSTCM,VISMSRSN_CODE,ASSESCMP,LGCMED2,LGAE2,LGCONDTN,COMM,ORIG_ENTRY,LAST_UPDATE,QUERY,SITE_APRV
0,233160901,V,3400,SC,SIG,07/2010,1,NaN,NaN,1.0,1.0,3.0,3.0,NaN,07/2010,2016-07-05 08:34:06.0,NaN,07/2010
1,232523801,V,3401,SC,SIG,07/2010,1,NaN,NaN,1.0,1.0,3.0,3.0,NaN,07/2010,2010-07-13 13:21:48.0,NaN,07/2010
2,233845301,V,3401,BL,SIG,07/2010,4,NaN,NaN,1.0,1.0,1.0,3.0,MR completed outside of window due to subject'...,07/2010,2010-09-21 08:29:06.0,NaN,07/2010
3,234863801,V,3400,BL,SIG,07/2010,4,NaN,NaN,1.0,2.0,3.0,3.0,Visit out of window due to subject's schedule,07/2010,2010-09-14 06:09:53.0,NaN,07/2010
4,236050301,V,3404,BL,SIG,07/2010,4,NaN,NaN,1.0,2.0,3.0,1.0,BL visit occurred out of window due to a scree...,07/2010,2010-09-21 08:41:48.0,NaN,07/2010


## Datasets with two few data points (not enough patients)

### IPSC_Labs: useful - Feat-bio: blood, liver disease, heparin(?), myeloma(?) stuff (linked to Med dose history)
### ONLY 137 patients...

In [30]:
### ISPC_Labs: useful - Feat-bio
#### Select only history of liver disease and multiple myeloma (can be useful for treatments strategies)
IPSCLAB=dict_files[files_names[5]]
#IPSCLAB.head()
IPSCLAB.columns

SEL=['REC_ID','PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PDMEDYN', 'PDMEDDT', 'PDMEDTM', 'WARFBEF', 'HEPBEF', 'HXLIVER',
       'HXMMYLMA', 'BLDLTH', 'LTHTM', 'LTHINVN', 'BLDSST', 'BLDCPT']
# we might be able to remove some features. 
print(number_of_patients(IPSCLAB))
IPSCLAB_sel=IPSCLAB[SEL]
IPSCLAB_sel.head()
#list(set(IPSCLAB_sel['EVENT_ID']))

79


KeyError: "['REC_ID' 'EVENT_ID' 'PAG_NAME' 'INFODT' 'PDMEDYN' 'PDMEDDT' 'PDMEDTM'\n 'WARFBEF' 'HEPBEF' 'HXLIVER' 'HXMMYLMA' 'BLDLTH' 'LTHTM' 'LTHINVN'\n 'BLDSST' 'BLDCPT'] not in index"

### 'Clinical_Diagnosis_and_Management.csv': Feat-PD - only 73 patients... (just for SWEDD). 

In [ ]:
### 'Clinical_Diagnosis_and_Management.csv': Feat-PD
CLINDX=dict_files[files_names[2]]
CLINDX.head()
#CLINDX.columns

SEL=['REC_ID', 'PATNO', 'EVENT_ID', 'PAG_NAME', 'INFODT',
       'PSLVL', 'DCNOMTR', 'DCRTREM', 'DCRIGID', 'DCBRADY', 'DFPGDIST',
       'DFOTH', 'PRIMDIAG', 'OTHNEURO', 'CLDXCHNG', 'DXFDTIMG',
       'DXFCLSGN', 'DXFRSPMD', 'DXFNATHX', 'DXFOTH', 'DXFOTHCM', 'MNGCHNG',
       'MNGTRTSX', 'MNGTRIAL', 'MNGTRTOT', 'MNGDXTST', 'MNGNOTRT',
       'NEURWKDX']
print(number_of_patients(CLINDX))
CLINDX_sel=CLINDX[SEL]
CLINDX_sel.head()
#list(set(CLINDX_sel['EVENT_ID']))

### 'TAP-PD_OPDM_Assessment.csv' - only 173 patients...


In [ ]:
TAPOPDM=dict_files[files_names[8]]
print(number_of_patients(TAPOPDM))
TAPOPDM.head()